# Objet : Test analysis

## Objectif

- tests des méthodes d'analyse


## Autres points


------
## Initialisation
- lecture des fichiers de 01/2022 issus de l'api (un fichier par jour)

In [1]:
from pprint import pprint
from collections import Counter
from time import time
import csv
import os
#os.chdir('C:/Users/a179227/OneDrive - Alliance/perso Wx/ES standard/python ESstandard/ES')
from util import util
from observation import Ilist, Iindex
from copy import copy

chemin='C:/Users/a179227/OneDrive - Alliance/perso Wx/ES standard/Environnemental-Sensing/python/validation/air/data_lcsqa/'

In [2]:
import pandas as pd
data = []
t0 = time()
annee = 2022
mois = 1
jour = 1
for i in range(4):
    file = chemin + 'FR_E2_' + str(annee) + '-' + format(mois, '02d') +'-' + format(jour+i, '02d') +'.csv'
    data.append(pd.read_csv(file, sep=';'))
data2 = pd.concat(data, ignore_index=True, join='inner').astype('category')
data2.pop('valeur brute')
print('data2', len(data2), list(data2), time()-t0)

data2 197455 ['Date de début', 'Date de fin', 'Organisme', 'code zas', 'Zas', 'code site', 'nom site', "type d'implantation", 'Polluant', "type d'influence", 'discriminant', 'Réglementaire', "type d'évaluation", 'procédure de mesure', 'type de valeur', 'valeur', 'unité de mesure', 'taux de saisie', 'couverture temporelle', 'couverture de données', 'code qualité', 'validité'] 1.6598665714263916


In [3]:
t0=time()
idxs2 = Ilist.obj(data2)
print('idxs (len, lenlidx, sumcodec) : ', len(idxs2), len(idxs2.idxlen), sum(idxs2.idxlen), time()-t0)
#idxs2.delindex('valeur brute')
idxs2.setvar('valeur')

idxs (len, lenlidx, sumcodec) :  197455 22 4462 1.8885533809661865


In [4]:
#calcul de values à partir de keys et codec
t0=time()
for idx in idxs2.lindex:
    val = idx.values
print('list to val old: ', time()-t0)
#génération d'une liste de Series à partir d'une liste de codec
t0=time()
pdcodec=[pd.Series(idx.codec) for idx in idxs2.lindex]
print('list to codec : ', time()-t0)#génération d'une liste de Series à partir d'une liste de keys
#génération d'une liste de Series simple à partir d'une liste de keys
t0=time()
pdkeyss=[pd.Series(idx.keys, dtype='int64') for idx in idxs2.lindex]
print('list to cat simple : ', time()-t0)
#génération d'une liste de Series à partir d'une liste de keys
t0=time()
pdkeys=[pd.Series(idx.keys, dtype='category') for idx in idxs2.lindex]
print('list to cat : ', time()-t0)
#génération d'une liste de Series à partir d'une liste de keys
pdkeys=[]
t0=time()
for idx in idxs2.lindex:
    pdkeys.append(pd.Series(idx.keys, dtype='category'))
print('list to cat : ', time()-t0)#calcul de values à partir de keys et codec avec une Series categorical (default)
t0=time()
for idx in idxs2.lindex:
    val = list(pd.Series(idx.keys, dtype='category').cat.rename_categories(idx.codec))
print('list to val (def): ', time()-t0)
#calcul de values à partir de keys et codec avec une Series map (all)
t0=time()
for idx in idxs2.lindex:
    val = list(pd.Series(idx.keys).map(pd.Series(idx.codec)))
print('list to val (all map): ', time()-t0)
#calcul de values à partir de keys et codec avec une Series (quelconque)
t0=time()
for idx, keys in zip(idxs2.lindex,pdkeys):
    val = list(pd.DataFrame(keys.cat.rename_categories(pd.Series(zip(idx.codec, range(len(idx.codec))))).tolist())[0])
print('list to val (all tuple): ', time()-t0)
#calcul de values à partir d'une Series de keys et codec (default)
t0=time()
for idx, keys in zip(idxs2.lindex,pdkeys):
    val = list(keys.cat.rename_categories(idx.codec))
print('cat to val (def): ', time()-t0)
#calcul de values à partir d'une Series de keys et codec avec une Series (quelconque)
t0=time()
for idx, keys in zip(idxs2.lindex,pdkeys):
    val = list(keys.map(pd.Series(idx.codec)))
print('cat to val (all): ', time()-t0)
#calcul de values à partir d'une Series de keys et une series codec (quelconque)
t0=time()
for cod, keys in zip(pdcodec,pdkeyss):
    val = list(keys.map(cod))
print('simple to val (all series simple): ', time()-t0)
#calcul de values à partir d'une Series de keys et une series codec (quelconque)
t0=time()
for cod, keys in zip(pdcodec,pdkeys):
    val = list(keys.map(cod))
print('cat to val (all series): ', time()-t0)

list to val old:  0.7261416912078857
list to codec :  0.0
list to cat simple :  0.6539871692657471
list to cat :  1.4012141227722168
list to cat :  1.4895117282867432
list to val (def):  1.6453883647918701
list to val (all map):  1.7189126014709473
list to val (all tuple):  2.1697654724121094
cat to val (def):  0.177293062210083
cat to val (all):  0.20064997673034668
simple to val (all series simple):  0.5261886119842529
cat to val (all series):  0.19498419761657715


In [5]:
#print(val[:100])

In [6]:
#ESValue
from observation import DatationValue, NamedValue
ser = pd.Series([DatationValue('date'), DatationValue('date2'), DatationValue('date'), NamedValue(3,'test')], dtype='category')
print(ser.cat.categories, '\n', list(ser.cat.codes))
ser2 = pd.Series([0,11,0,22]).astype('category')
serbis = ser2.cat.rename_categories([DatationValue('date'), DatationValue('date2'),  NamedValue(3,'test')])
min(ser.astype('object') == serbis.astype('object'))

Index([{"test": 3}, "date", "date2"], dtype='object') 
 [1, 2, 1, 0]


True

In [7]:
#generation dataframe à partir de Ilist
ilm = Ilist.obj([['plants', ['fruit', 'fruit', 'fruit', 'fruit', 'vegetable', 'vegetable', 'vegetable', 'fruit']],
                  ['quantity', ['kg', '10 kg', 'kg', '10 kg', 'kg', '10 kg', 'kg', '10 kg']],
                  ['product', ['apple', 'apple', 'orange', 'orange', 'peppers', 'peppers', 'banana', 'banana']],
                  ['price', [1, 10, 2, 20, 1.5, 15, 0.5, 5], -1]])
df = pd.concat([idx.to_pandas(index=False, series=False).astype('category') for idx in ilm.lindex], axis=1)
col = pd.MultiIndex.from_arrays([ilm.lname, ilm.lisvar], names=['name', 'var'])
df.columns = col
print(df)

name     plants quantity  product price
var       False    False    False  True
0         fruit       kg    apple   1.0
1         fruit    10 kg    apple  10.0
2         fruit       kg   orange   2.0
3         fruit    10 kg   orange  20.0
4     vegetable       kg  peppers   1.5
5     vegetable    10 kg  peppers  15.0
6     vegetable       kg   banana   0.5
7         fruit    10 kg   banana   5.0


In [8]:
# fonction append
s2 = pd.Series(['fruit', 'kg', 'banana', 7], index=df.columns)
df = pd.concat([df, s2.to_frame().T], ignore_index=True).astype('category') 
ds = df['price'].squeeze()
ds.cat.categories

Float64Index([0.5, 1.0, 1.5, 2.0, 5.0, 7.0, 10.0, 15.0, 20.0], dtype='float64')

In [9]:
a= pd.Series(['b', 'a', 'a', 'd']).astype('category')
b = pd.Series([1,2,4,3]).astype('category')
coup = pd.Series(list(zip(*(list(a), list(b))))).astype('category')
print(coup)

print(a.cat.codes)
print(coup.cat.codes)
res = coup.cat.codes == a.cat.codes
print(res)

0    (b, 1)
1    (a, 2)
2    (a, 4)
3    (d, 3)
dtype: category
Categories (4, object): [('a', 2), ('a', 4), ('b', 1), ('d', 3)]
0    1
1    0
2    0
3    2
dtype: int8
0    2
1    0
2    1
3    3
dtype: int8
0    False
1     True
2    False
3    False
dtype: bool


In [10]:
a = pd.Series([0,1,2,3,2]).astype('category')
codec = ['a', 'b', 'c', 'b']
print(a.cat.codes.map(pd.Series(codec)))

0    a
1    b
2    c
3    b
4    c
dtype: object


In [11]:
#filtrage suivant var
df.loc[:,(slice(None),False)]

name,plants,quantity,product
var,False,False,False
0,fruit,kg,apple
1,fruit,10 kg,apple
2,fruit,kg,orange
3,fruit,10 kg,orange
4,vegetable,kg,peppers
5,vegetable,10 kg,peppers
6,vegetable,kg,banana
7,fruit,10 kg,banana
8,fruit,kg,banana


In [12]:
full = [[name, list(ds)] for name,ds in df.items()]
print(full, '\n')
default = [[name, list(ds.cat.categories), list(ds.cat.codes)] for name,ds in df.astype('category').items()]
print(default)

[[('plants', False), ['fruit', 'fruit', 'fruit', 'fruit', 'vegetable', 'vegetable', 'vegetable', 'fruit', 'fruit']], [('quantity', False), ['kg', '10 kg', 'kg', '10 kg', 'kg', '10 kg', 'kg', '10 kg', 'kg']], [('product', False), ['apple', 'apple', 'orange', 'orange', 'peppers', 'peppers', 'banana', 'banana', 'banana']], [('price', True), [1.0, 10.0, 2.0, 20.0, 1.5, 15.0, 0.5, 5.0, 7.0]]] 

[[('plants', False), ['fruit', 'vegetable'], [0, 0, 0, 0, 1, 1, 1, 0, 0]], [('quantity', False), ['10 kg', 'kg'], [1, 0, 1, 0, 1, 0, 1, 0, 1]], [('product', False), ['apple', 'banana', 'orange', 'peppers'], [0, 0, 2, 2, 3, 3, 1, 1, 1]], [('price', True), [0.5, 1.0, 1.5, 2.0, 5.0, 7.0, 10.0, 15.0, 20.0], [1, 6, 3, 8, 2, 7, 0, 4, 5]]]


In [13]:
print(ilm.to_obj(modecodec='full'), '\n')
print(ilm.to_obj(modecodec='default'))

[['plants', ['fruit', 'fruit', 'fruit', 'fruit', 'vegetable', 'vegetable', 'vegetable', 'fruit']], ['quantity', ['kg', '10 kg', 'kg', '10 kg', 'kg', '10 kg', 'kg', '10 kg']], ['product', ['apple', 'apple', 'orange', 'orange', 'peppers', 'peppers', 'banana', 'banana']], ['price', [1, 10, 2, 20, 1.5, 15, 0.5, 5], -1]] 

[['plants', ['vegetable', 'fruit'], [1, 1, 1, 1, 0, 0, 0, 1]], ['quantity', ['kg', '10 kg'], [0, 1, 0, 1, 0, 1, 0, 1]], ['product', ['banana', 'peppers', 'orange', 'apple'], [3, 3, 2, 2, 1, 1, 0, 0]], ['price', [1, 10, 2, 20, 1.5, 15, 0.5, 5], -1]]


In [14]:
#generation de json (full et default)
import json
t0=time()
full = json.dumps([[name, list(ds)] for name,ds in data2.items()])
print('fullsize', len(full), time()-t0)
t0=time()
default = json.dumps([[name, list(ds.cat.categories), list(ds.cat.codes)] for name,ds in data2.astype('category').items()])
print('defaultsize', len(default), time()-t0)


fullsize 58111978 0.97560715675354
defaultsize 15847186 1.0072519779205322


In [15]:
from pprint import pformat

def dic_noeud(n):
    lis = [noeud[n+1][0] + ' - ' + str(noeud[n+1][2])]
    if child[n+1]:
        for ch in child[n+1]:
            if ch != n:
                lis.append(dic_noeud(ch))
    return {n: lis}
    #return {noeud[n+1][1]: lis}

noeud = [['root', -1, 5, -1], ['six',0 , 4, -1], ['deux', 1, 0, 0], ['trois', 2, 1, 1], 
         ['quatre', 3, 25, 1], ['cinq', 4 , 3, 0], ['premier', 5, 4, -1]] 
child = [None] * len(noeud)
for n in noeud: 
    if child[n[3]+1] is None:
        child[n[3]+1] = []
    child[n[3]+1].append(n[1])
    #print(child)
print(child)
tre = dic_noeud(-1)
print(tre)

#tre={'root':[5 ,{'six':[4,{'deux':[0,{'trois':[1]}, {'quatre':[25]}]}, {'cinq':[3]}]}, {'premier':[4]}]}
tref = pformat(tre, indent=0, width=1)
for c in ["'", "{", "[", "]", "}", ","]: tref = tref.replace (c, "")
print(tref)   

[[-1, 0, 5], [1, 4], [2, 3], None, None, None, None]
{-1: ['root - 5', {0: ['six - 4', {1: ['deux - 0', {2: ['trois - 1']}, {3: ['quatre - 25']}]}, {4: ['cinq - 3']}]}, {5: ['premier - 4']}]}
-1: root 
    - 
    5
    0: six 
       - 
       4
       1: deux 
          - 
          0
          2: trois 
             - 
             1
          3: quatre 
             - 
             25
       4: cinq 
          - 
          3
    5: premier 
       - 
       4


In [16]:
t0=time()
fullsize = len(idxs2.to_obj(encoded=True, modecodec='full'))
print('fullsize', fullsize, time()-t0)
t0=time()
minsize = len(idxs2.to_obj(encoded=True, modecodec='nokeys'))
print('minsize', minsize, time()-t0)

fullsize 57534721 10.840340614318848
minsize 1136118 0.6176717281341553


----
## format non optimisé
- le "taux d'unicité" reste à 12% (pas de modification des index)
- le "taux de codage" est de 30% (remplacement des données dupliquées par un entier)
- le gain de taille de fichier par rapport à un fichier "quoté" est de 61%
- l'analyse de la structure montre que les données sont principalement du type "linked" (non ou peu structuré)
- quelques colonnes sont de type "derived". Par exemple les index longitude(43) et latitude(44) sont bien dérivés de l'index coordonneesXY(13)
- le taux de couplage ("linkrate") pour chacun des index est très proche de 0, ce qui signifie que les données devraient être de type "derived" (lien de dépendance par exemple comme entre les trimestres et les mois) ou "coupled" (lien biunivoque).

In [17]:
t0=time()
defaultsize = len(idxs2.to_obj(encoded=True, modecodec='default'))
print('defaultsize', defaultsize, time()-t0)
print('indicator default : ', idxs2.indicator(fullsize, defaultsize))
pprint(idxs2.indexinfos(keys=['num', 'name', 'lencodec', 'parent', 'typecoupl']), width=120)
pprint(idxs2.indexinfos(keys=['linkrate']))

defaultsize 15345438 0.9345924854278564
indicator default :  {'init values': 4344010, 'mean size': 13.245, 'unique values': 1557, 'mean coding size': 3.529, 'unicity level': 0.0, 'object lightness': 0.266, 'gain': 0.733}
[{'lencodec': 96, 'name': 'Date de début', 'num': 0, 'parent': 5, 'typecoupl': 'linked'},
 {'lencodec': 96, 'name': 'Date de fin', 'num': 1, 'parent': 0, 'typecoupl': 'coupled'},
 {'lencodec': 18, 'name': 'Organisme', 'num': 2, 'parent': 5, 'typecoupl': 'derived'},
 {'lencodec': 70, 'name': 'code zas', 'num': 3, 'parent': 5, 'typecoupl': 'derived'},
 {'lencodec': 70, 'name': 'Zas', 'num': 4, 'parent': 3, 'typecoupl': 'coupled'},
 {'lencodec': 533, 'name': 'code site', 'num': 5, 'parent': 5, 'typecoupl': 'crossed'},
 {'lencodec': 533, 'name': 'nom site', 'num': 6, 'parent': 5, 'typecoupl': 'linked'},
 {'lencodec': 5, 'name': "type d'implantation", 'num': 7, 'parent': 5, 'typecoupl': 'derived'},
 {'lencodec': 9, 'name': 'Polluant', 'num': 8, 'parent': 13, 'typecoupl': 'l

----
## Format optimisé
- le "taux d'unicité" se dégrade légèrement (passage de 11,6% à 12,1%) par l'ajout d'index supplémentaires
- le "taux de codage" par contre passe de 30% à 16% de par l'optimisation 
- le gain de taille de fichier par rapport à un fichier "quoté" est maintenant de 74%
- l'utilisation d'un format binaire (codage CBOR pour Concise Binary Object Representation RFC 8949) permet d'améliorer encore le gain de taille de fichier (82%)

In [18]:
#idxs.setcanonorder().sort()
t0=time()
optimizesize = len(idxs2.to_obj(modecodec='optimize', encoded=True))
print('optimizesize ', optimizesize, time()-t0)
print('indicator optimize : ', idxs2.indicator(fullsize, optimizesize))
t0=time()
js = idxs2.to_obj(encoded=True, modecodec='optimize', encode_format='cbor')
cborsize = len(js)
print('cborsize', cborsize, time()-t0)
print('indicator cbor : ', idxs2.indicator(fullsize, cborsize))

optimizesize  8831886 1.5437443256378174
indicator optimize :  {'init values': 4344010, 'mean size': 13.245, 'unique values': 1557, 'mean coding size': 2.029, 'unicity level': 0.0, 'object lightness': 0.153, 'gain': 0.846}
cborsize 4608236 1.2426013946533203
indicator cbor :  {'init values': 4344010, 'mean size': 13.245, 'unique values': 1557, 'mean coding size': 1.056, 'unicity level': 0.0, 'object lightness': 0.08, 'gain': 0.92}


----
## Intégrité
- la transformation inverse des données binaires permet de vérifier qu'on retombe bien sur les mêmes données (pas de dégradation)

In [19]:
t0=time()
idxs3 = Ilist.from_obj(js)
print('fromcbor', len(idxs3), time()-t0)
t0=time()
verif = idxs2 == idxs3
print('controle égalité :', verif, time()-t0)


fromcbor 197455 2.801172971725464
controle égalité : True 1.4496169090270996
